<a href="https://colab.research.google.com/github/igorcabralbr/FipeVeiculos/blob/main/FipeVeiculos_FranckePeixoto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://www.fipe.org.br/Content/img/logo_fipe.png' />

veiculos.fipe.org.br



In [89]:
#@title FipeVeiculos v1
#@markdown Consulta de Carros e Utilitários Pequenos
import requests
import pandas as pd
class FipeVeiculos:
  #------------------
  #
  # FipeVeiculos v1 - Consulta de Carros e Utilitários Pequenos
  # Informe primeiro a marca do veículo e depois o modelo e o ano modelo na ordem que desejar.
  # Você também pode digitar a informação no campo "busca", dentro de cada opção do formulário da pesquisa.
  # https://veiculos.fipe.org.br/api/veiculos/
  #
  # By Francke Peixoto
  #-------------------------------

  def __init__(self,codigoTipoVeiculo = 1):

    self.url_base = "https://veiculos.fipe.org.br/api/veiculos/"
    self.headers  = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36","From": "youremail@domain.com"}
    self.codigoTipoVeiculo = codigoTipoVeiculo
    self.tabelaReferencia = None
    self.marcas = None
    self.modelos = None
    self.tabela = {};
    self.tabelaAnoModelo = {};

  def ObterTabelaDeReferencia(self):

    if self.tabelaReferencia is None:
      response  = requests.post(self.url_base+'ConsultarTabelaDeReferencia', headers = self.headers)
      self.headers = requests.utils.default_headers()
      self.tabelaReferencia = response.json()
      self.tabelaReferencia = pd.DataFrame(data=self.tabelaReferencia)
    return self.tabelaReferencia

  def ObterMarcas(self,codigoTabelaReferencia):

    if self.marcas is None:
      payload = {}
      payload['codigoTipoVeiculo'] = self.codigoTipoVeiculo
      payload['codigoTabelaReferencia'] = codigoTabelaReferencia
      response  = requests.post(self.url_base+'ConsultarMarcas',payload, headers = self.headers)
      headers = requests.utils.default_headers()
      self.marcas = response.json()
      self.marcas = pd.DataFrame(data=self.marcas)
    return self.marcas

  def ObterModelos(self,codigoTabelaReferencia,codigoMarca):

    if self.modelos is None:
      payload = {}
      payload['codigoTipoVeiculo'] = self.codigoTipoVeiculo
      payload['codigoTabelaReferencia'] = codigoTabelaReferencia
      payload['codigoMarca'] = codigoMarca
      response  = requests.post(self.url_base+'ConsultarModelos',payload, headers = self.headers)
      headers = requests.utils.default_headers()
      self.modelos = response.json()['Modelos']
    return pd.DataFrame(data=self.modelos)

  def ObterAnoModelo(self,codigoTabelaReferencia,codigoMarca,codigoModelo):

    chave = str(codigoTabelaReferencia)+'_'+str(codigoMarca)+'_'+str(codigoModelo)
    if self.tabelaAnoModelo.get(chave) is None:
      payload = {}
      payload['codigoTipoVeiculo'] = self.codigoTipoVeiculo
      payload['codigoTabelaReferencia'] = codigoTabelaReferencia
      payload['codigoMarca'] = codigoMarca
      payload['codigoModelo'] = codigoModelo
      response  = requests.post(self.url_base+'ConsultarAnoModelo',payload, headers = self.headers)
      headers = requests.utils.default_headers()
      self.anoModelos = response.json()
      self.anoModelos = pd.DataFrame(data=self.anoModelos)
      self.anoModelos['Value'] = self.anoModelos['Value'].transform(lambda v: v.split('-')[0])
      self.tabelaAnoModelo[chave] = self.anoModelos
    return self.tabelaAnoModelo[chave]

  def __ajustarValores(self, rows):

    carros = pd.DataFrame(data=rows)
    carros.Valor = carros.Valor.replace('[R$]', '', regex=True).replace('[\.]', '', regex=True).replace('[\,]', '.', regex=True).astype(float)
    carros.AnoModelo = carros.AnoModelo.astype(int)
    dropColumns = ['TipoVeiculo','DataConsulta','Autenticacao']
    carros.drop(columns=dropColumns,inplace=True)
    return carros

  def ObterValores(self, codigoMarca,codigoModelo,limite=24):

    chave = str(codigoMarca)+'_'+str(codigoModelo)
    if self.tabela.get(chave) is None:
      self.ObterTabelaDeReferencia()
      self.rows =[]
      for referencia in  self.tabelaReferencia.values[:limite]:
        codigoTabelaReferencia = referencia[0]

        anoModelos = self.ObterAnoModelo(codigoTabelaReferencia,codigoMarca,codigoModelo)

        print(referencia)
        print(codigoTabelaReferencia)
        print(self.tabelaReferencia.values[:limite])

        limite = limite-4

        payload = {}
        payload['codigoTabelaReferencia'] = codigoTabelaReferencia
        payload['codigoMarca'] = codigoMarca
        payload['codigoModelo'] = codigoModelo
        payload['codigoTipoVeiculo'] = self.codigoTipoVeiculo
        payload['anoModelo'] = 1998 #jim
        payload['codigoTipoCombustivel'] =1
        payload['tipoVeiculo']= 'carro'
        payload['modeloCodigoExterno'] =''
        payload['tipoConsulta'] = 'tradicional'
        response  = requests.post(self.url_base+'ConsultarValorComTodosParametros',payload, headers = self.headers)
        headers = requests.utils.default_headers()
        row = response.json()
        if len(row) > 9:
          self.rows.append(row)

      self.tabela[chave] = self.__ajustarValores(self.rows)
    return self.tabela[chave]


In [90]:
fipe = FipeVeiculos()
codigoTabelaReferencia  = fipe.ObterTabelaDeReferencia()[:1].Codigo.values[0]
print(fipe.ObterTabelaDeReferencia()[:1],' >>>> ',codigoTabelaReferencia)

   Codigo          Mes
0     310  junho/2024   >>>>  310


In [91]:
marcas = fipe.ObterMarcas(codigoTabelaReferencia)
nomes =['Honda']
marcas[marcas.Label.str.strip().isin(nomes)].head()

,Label,Value
43,Honda,25


In [92]:
fipe.ObterTabelaDeReferencia().head()

,Codigo,Mes
0,310,junho/2024
1,309,maio/2024
2,308,abril/2024
3,307,março/2024
4,306,fevereiro/2024


In [93]:
codigoMarca = 25
modelos = fipe.ObterModelos(codigoTabelaReferencia,codigoMarca)
modelos

,Label,Value
0,Accord Coupe EX,1241
1,Accord Sedan 2.0 TB 16V Aut. (Híbrido),9600
2,Accord Sedan EX 2.0 16V 156cv Aut.,5211
3,Accord Sedan EX 2.4/2.3/ 2.2 16V,1242
4,Accord Sedan EX 2.7/3.0 24V,1243
...,...,...
111,Prelude Si,1285
112,WR-V EX 1.5 Flexone 16V 5p Aut.,7899
113,WR-V EXL 1.5 Flexone 16V 5p Aut.,7900
114,WR-V LX 1.5 Flexone 16V 5p Aut.,9195


In [94]:
codigoModelo = 1248
carros = fipe.ObterValores(codigoMarca,codigoModelo, limite=12)
carros.sort_values(['AnoModelo', 'MesReferencia'], ascending = [True, True])
carros.to_csv('carros.csv')
carros

[310 'junho/2024 ']
310
[[310 'junho/2024 ']
 [309 'maio/2024 ']
 [308 'abril/2024 ']
 [307 'março/2024 ']
 [306 'fevereiro/2024 ']
 [305 'janeiro/2024 ']
 [304 'dezembro/2023 ']
 [303 'novembro/2023 ']
 [302 'outubro/2023 ']
 [301 'setembro/2023 ']
 [300 'agosto/2023 ']
 [299 'julho/2023 ']]
[309 'maio/2024 ']
309
[[310 'junho/2024 ']
 [309 'maio/2024 ']
 [308 'abril/2024 ']
 [307 'março/2024 ']
 [306 'fevereiro/2024 ']
 [305 'janeiro/2024 ']
 [304 'dezembro/2023 ']
 [303 'novembro/2023 ']]
[308 'abril/2024 ']
308
[[310 'junho/2024 ']
 [309 'maio/2024 ']
 [308 'abril/2024 ']
 [307 'março/2024 ']]
[307 'março/2024 ']
307
[]
[306 'fevereiro/2024 ']
306
[[310 'junho/2024 ']
 [309 'maio/2024 ']
 [308 'abril/2024 ']
 [307 'março/2024 ']
 [306 'fevereiro/2024 ']
 [305 'janeiro/2024 ']
 [304 'dezembro/2023 ']
 [303 'novembro/2023 ']
 [302 'outubro/2023 ']
 [301 'setembro/2023 ']
 [300 'agosto/2023 ']
 [299 'julho/2023 ']
 [298 'junho/2023 ']
 [297 'maio/2023 ']
 [296 'abril/2023 ']
 [295 'ma

,Valor,Marca,Modelo,AnoModelo,Combustivel,CodigoFipe,MesReferencia,SiglaCombustivel
0,28304.0,Honda,Civic Coupe EX/ EXS 1.6 16V 2p,1998,Gasolina,014019-8,junho de 2024,G
1,27750.0,Honda,Civic Coupe EX/ EXS 1.6 16V 2p,1998,Gasolina,014019-8,maio de 2024,G
2,27395.0,Honda,Civic Coupe EX/ EXS 1.6 16V 2p,1998,Gasolina,014019-8,abril de 2024,G
3,27210.0,Honda,Civic Coupe EX/ EXS 1.6 16V 2p,1998,Gasolina,014019-8,março de 2024,G
4,26678.0,Honda,Civic Coupe EX/ EXS 1.6 16V 2p,1998,Gasolina,014019-8,fevereiro de 2024,G
5,26157.0,Honda,Civic Coupe EX/ EXS 1.6 16V 2p,1998,Gasolina,014019-8,janeiro de 2024,G
6,25951.0,Honda,Civic Coupe EX/ EXS 1.6 16V 2p,1998,Gasolina,014019-8,dezembro de 2023,G
7,25691.0,Honda,Civic Coupe EX/ EXS 1.6 16V 2p,1998,Gasolina,014019-8,novembro de 2023,G
8,25188.0,Honda,Civic Coupe EX/ EXS 1.6 16V 2p,1998,Gasolina,014019-8,outubro de 2023,G
9,24999.0,Honda,Civic Coupe EX/ EXS 1.6 16V 2p,1998,Gasolina,014019-8,setembro de 2023,G


In [95]:
print(carros.to_string())

      Valor  Marca                          Modelo  AnoModelo Combustivel CodigoFipe       MesReferencia SiglaCombustivel
0   28304.0  Honda  Civic Coupe EX/ EXS 1.6 16V 2p       1998    Gasolina   014019-8      junho de 2024                 G
1   27750.0  Honda  Civic Coupe EX/ EXS 1.6 16V 2p       1998    Gasolina   014019-8       maio de 2024                 G
2   27395.0  Honda  Civic Coupe EX/ EXS 1.6 16V 2p       1998    Gasolina   014019-8      abril de 2024                 G
3   27210.0  Honda  Civic Coupe EX/ EXS 1.6 16V 2p       1998    Gasolina   014019-8      março de 2024                 G
4   26678.0  Honda  Civic Coupe EX/ EXS 1.6 16V 2p       1998    Gasolina   014019-8  fevereiro de 2024                 G
5   26157.0  Honda  Civic Coupe EX/ EXS 1.6 16V 2p       1998    Gasolina   014019-8    janeiro de 2024                 G
6   25951.0  Honda  Civic Coupe EX/ EXS 1.6 16V 2p       1998    Gasolina   014019-8   dezembro de 2023                 G
7   25691.0  Honda  Civi